In [1]:
!pip install -q flwr
!pip install -U ipywidgets
!pip install -U flwr["simulation"]

In [2]:
import medmnist
from medmnist import INFO, Evaluator
from torchvision import transforms
from torchvision.transforms import ToTensor, Lambda
import torch.utils.data as data
from torchvision import models
import torch
import tqdm
from collections import OrderedDict
from typing import List, Tuple
import numpy as np 
from torch.utils.data import DataLoader, random_split
import flwr as fl
from torch import nn
import torch.nn.functional as F
from flwr.common import Metrics


DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


## Dataset Loading

In [3]:
# def load_data(batch_size=64):
#     data_transform = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[.5], std=[.5])
#     ])
#     data_flag = 'chestmnist'
#     info = INFO[data_flag]
#     DataClass = getattr(medmnist, info['python_class'])

#     train_set = DataClass(split='train', transform=data_transform, target_transform=Lambda(lambda y: y[0]), download=True)
#     test_set = DataClass(split='test', transform=data_transform, target_transform=Lambda(lambda y: y[0]), download=True)
#     # val_set = DataClass(split='val', transform=data_transform, target_transform=Lambda(lambda y: y[0]), download=True)
    
#     train_loader = data.DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
#     test_loader = data.DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)
#     # valid_loader = data.DataLoader(dataset=val_set, batch_size=batch_size, shuffle=False)

#     num_examples = {"trainset" : len(train_set), "testset" : len(test_set)}
#     return train_loader, test_loader, num_examples


def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[.5], std=[.5])
    ])
    # transform = transforms.Compose(
    #     [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    # )
    data_flag = 'chestmnist'
    info = INFO[data_flag]
    DataClass = getattr(medmnist, info['python_class'])

    trainset = DataClass(split='train', transform=transform, target_transform=Lambda(lambda y: y[0]), download=True)
    testset = DataClass(split='test', transform=transform, target_transform=Lambda(lambda y: y[0]), download=True)
    valset = DataClass(split='val', transform=transform, target_transform=Lambda(lambda y: y[0]), download=True)

    #  Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    if partition_size * num_clients == len(valset):
        lengths = [partition_size] * num_clients 
    else: 
         lengths = [partition_size] * (num_clients-1)  + [partition_size+ (len(trainset) % num_clients)]
    train_datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))
    
    partition_size = len(valset) // num_clients
    if partition_size * num_clients == len(valset):
        lengths = [partition_size] * num_clients 
    else: 
         lengths = [partition_size] * (num_clients-1)  + [partition_size+ (len(valset) % num_clients)]
    # print(len(valset), lengths)
    val_datasets = random_split(valset, lengths, torch.Generator().manual_seed(42))
    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []

    for t_ds in train_datasets:
        # len_train = len(t_ds) 
        # t_lengths = [len_train]
        # ds_train = random_split(t_ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(t_ds, batch_size=32, shuffle=True))


    for v_ds in val_datasets:
        # len_val = len(v_ds) // 10  # 10 % validation set
        # v_lengths = [len_val]
        # ds_val = random_split(v_ds, lengths, torch.Generator().manual_seed(42))
        valloaders.append(DataLoader(v_ds, batch_size=32))
        
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader



## Define Train and Test

In [11]:
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""

    net.train()
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    accuracy, loss = 0, 0
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for i, batch in enumerate(trainloader):
            # print(i)
            images, labels = batch
            optimizer.zero_grad()
            images = images.expand(-1, 3, -1, -1)
            outputs = net(images)
            labels = labels.long()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            predicted = torch.argmax(outputs.detach().cpu(), axis=1)
            correct += (predicted == labels).sum().item()
            # correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        # if verbose:
        # print(f"Epoch {epoch}: train loss {epoch_loss}, accuracy {epoch_acc}")
    return epoch_loss, epoch_acc 

def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch
            images = images.expand(-1, 3, -1, -1)
            labels = labels.long()
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            # print(outputs)
            # _, predicted = torch.max(outputs.data, 1)
            predicted = torch.argmax(outputs.detach().cpu(), axis=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy            

## Model 

In [5]:

class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # print(x.shape)
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load model and data
# net = Net().to(DEVICE)

In [20]:
# class MedmnistClient(fl.client.NumPyClient):
#     def get_parameters(self, config):
#         return [val.cpu().numpy() for _, val in model.state_dict().items()]

#     def set_parameters(self, parameters):
#         params_dict = zip(model.state_dict().keys(), parameters)
#         state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
#         model.load_state_dict(state_dict, strict=True)

#     def fit(self, parameters, config):
#         self.set_parameters(parameters)
#         train(model, train_loader, epochs=1)
#         return self.get_parameters(config={}), num_examples["trainset"], {}

#     def evaluate(self, parameters, config):
#         self.set_parameters(parameters)
#         loss, accuracy = test(model, test_loader)
#         return float(loss), num_examples["testset"], {"accuracy": float(accuracy)}
    
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {"loss": float(loss), "accuracy": float(accuracy)}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"loss": float(loss), "accuracy": float(accuracy)}

def client_fn(cid) -> FlowerClient:
    # net = models.resnet18(weights='DEFAULT').to(DEVICE)
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

In [21]:

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [23]:
# Create an instance of the model and get the parameters
NUM_CLIENTS = 3
trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)
# params = get_parameters(models.resnet18(weights='DEFAULT'))
params = get_parameters(Net())
server_config = fl.server.ServerConfig(num_rounds=5)
# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=0.5,
    min_fit_clients=NUM_CLIENTS,
    min_evaluate_clients=2,
    min_available_clients=NUM_CLIENTS,
    # initial_parameters=fl.common.ndarrays_to_parameters(params),
    evaluate_metrics_aggregation_fn=weighted_average, 
    # fit_metrics_aggregation_fn=weighted_average
    
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=server_config,  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

Using downloaded and verified file: C:\Users\Sama\.medmnist\chestmnist.npz
Using downloaded and verified file: C:\Users\Sama\.medmnist\chestmnist.npz


INFO flwr 2024-01-18 15:19:01,294 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)


Using downloaded and verified file: C:\Users\Sama\.medmnist\chestmnist.npz


2024-01-18 15:19:06,354	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-01-18 15:19:10,504 | app.py:213 | Flower VCE: Ray initialized with resources: {'CPU': 16.0, 'object_store_memory': 2310544588.0, 'node:127.0.0.1': 1.0, 'memory': 4621089179.0, 'node:__internal_head__': 1.0}
INFO flwr 2024-01-18 15:19:10,504 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-01-18 15:19:10,505 | app.py:227 | No `client_resources` specified. Using minimal resources for clients.
INFO flwr 2024-01-18 15:19:10,506 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2024-01-18 15:19:10,524 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 16 actors
INFO flwr 2024-01-18 15:19:10,525 | server.py:89 | Initializing global parameters
INFO flwr 2024-01-18 15:19:10,526 | server.py:276 | Requesting initial parameters from one random client

(DefaultActor pid=20584) [Client 1] get_parameters
(DefaultActor pid=20584) [Client 1] fit, config: {}


...

(DefaultActor pid=20584) [Client 2] evaluate, config: {} [repeated 2x across cluster]
(DefaultActor pid=23512) [Client 2] fit, config: {} [repeated 3x across cluster]


DEBUG flwr 2024-01-18 15:20:22,198 | server.py:187 | evaluate_round 4 received 2 results and 0 failures
DEBUG flwr 2024-01-18 15:20:22,199 | server.py:222 | fit_round 5: strategy sampled 3 clients (out of 3)
DEBUG flwr 2024-01-18 15:20:35,771 | server.py:236 | fit_round 5 received 3 results and 0 failures
DEBUG flwr 2024-01-18 15:20:35,775 | server.py:173 | evaluate_round 5: strategy sampled 2 clients (out of 3)


(DefaultActor pid=20584) [Client 2] evaluate, config: {} [repeated 2x across cluster]
(DefaultActor pid=5908) [Client 2] fit, config: {} [repeated 3x across cluster]


DEBUG flwr 2024-01-18 15:20:36,933 | server.py:187 | evaluate_round 5 received 2 results and 0 failures
INFO flwr 2024-01-18 15:20:36,934 | server.py:153 | FL finished in 77.87219379999988
INFO flwr 2024-01-18 15:20:36,934 | app.py:226 | app_fit: losses_distributed [(1, 0.009619691928509373), (2, 0.009544501427445404), (3, 0.009243369295023372), (4, 0.009456421733679906), (5, 0.009161372952273405)]
INFO flwr 2024-01-18 15:20:36,935 | app.py:227 | app_fit: metrics_distributed_fit {'accuracy': [(1, 0.8968369271550186), (2, 0.8980985879594228), (3, 0.898098587959423), (4, 0.8980985879594228), (5, 0.8980985879594228)]}
INFO flwr 2024-01-18 15:20:36,935 | app.py:228 | app_fit: metrics_distributed {'accuracy': [(1, 0.9017116876170098), (2, 0.9021403172910518), (3, 0.9021403172910518), (4, 0.8969250190829748), (5, 0.9021403172910518)]}
INFO flwr 2024-01-18 15:20:36,936 | app.py:229 | app_fit: losses_centralized []
INFO flwr 2024-01-18 15:20:36,936 | app.py:230 | app_fit: metrics_centralized {

History (loss, distributed):
	round 1: 0.009619691928509373
	round 2: 0.009544501427445404
	round 3: 0.009243369295023372
	round 4: 0.009456421733679906
	round 5: 0.009161372952273405
History (metrics, distributed, fit):
{'accuracy': [(1, 0.8968369271550186), (2, 0.8980985879594228), (3, 0.898098587959423), (4, 0.8980985879594228), (5, 0.8980985879594228)]}History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.9017116876170098), (2, 0.9021403172910518), (3, 0.9021403172910518), (4, 0.8969250190829748), (5, 0.9021403172910518)]}

In [ ]:
# model = Net()
# train(model , trainloaders[0], epochs=3)
# loss, accuracy = test(model, valloaders[0])

In [ ]:
# fl.client.start_numpy_client(server_address="[::]:8080", client=MedmnistClient())